In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from utils.PredictSalesUtils import cleanStackingDataset

In [ ]:
import gc

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib

In [ ]:
import sys
sys.path.append('../dsbase/src/main')

In [ ]:
from sklearn.model_selection import train_test_split
from dsbase.ModelDSBase import ModelDSBaseWrapper

In [ ]:
from dsbase.models.regression.LightGradientBoostingRegressionDSBase import LightGradientBoostingRegressionDSBaseModel
from dsbase.models.regression.LightGradientBoostingRegressionDSBase import LightGradientBoostingRegressionDSBaseModelParamsToMap

In [ ]:
from dsbase.models.regression.DNNRegressionKerasDSBase import DNNRegressionKerasDSBaseModel
from dsbase.models.regression.DNNRegressionKerasDSBase import DNNRegressionKerasDSBaseParamsToMap

# Ensembling - Stacking

## Phase 1 (LightGB for each 1/2 dataset)

In [ ]:
def getXy(databaseName):
    df = pd.read_csv('datasets/' + databaseName + '_cleaned.csv')
    df.drop(['Unnamed: 0','Unnamed: 0.1','slot'], inplace=True, axis=1)
    df.set_index(keys='ID_pair',inplace=True)
    
    X = df.drop(labels=['target_imputed'], axis=1).values
    y = df['target_imputed'].values
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    return X_train, X_test, y_train, y_test

In [ ]:
# params: [reg_alpha, reg_lambda]
def processModel(X_train, X_test, y_train, y_test, modelName, params, perc=[100]):
    print('generating model',modelName,'...')

    params = LightGradientBoostingRegressionDSBaseModelParamsToMap(max_depth=31, n_estimators=100, learning_rate=0.1,num_leaves=31, subsample_for_bin=200000, reg_alpha=params[0], reg_lambda=params[1])
    lgbr = ModelDSBaseWrapper(modelName,X_train,y_train,X_test, y_test,perc,LightGradientBoostingRegressionDSBaseModel,params)
    lgbr.train()
    lclgdbr=lgbr.getLearningCurves()
    
    overfitting_rate = lclgdbr[1,-1]/lclgdbr[0,-1]
    score = lgbr.getScore()
    print('Score for',modelName,lgbr.getScore(),'(',overfitting_rate,')')
    return lgbr, overfitting_rate, score, lclgdbr

Let's calculate the models: 

In [ ]:
# Completed Set
dataset = ['dataset1A','dataset2A','dataset3A','dataset4A','dataset5A','dataset6A']
model = ['model1A','model2A','model3A','model4A','model5A','model6A']
params = [[3000,25],[2500,25],[2000,15],[1000,15],[100,10],[0,10]]
#scores = [0.7114320203335709 ( 0.9036347645739408 ), 0.7017113534310703 ( 0.8691804015123732 ), 0.6440095888038557 ( 0.8363979013208283 )
#          0.5942881438165505 ( 0.7507641992157429 ), 0.5763562599461289 ( 0.8655557217963179 ), 0.5480502465067804 ( 0.9972860257205527 )]
perc = [100]

# Searching params Set
#dataset = ['dataset4A']
#model = ['model4A']
#params = [[5000,50]]
#perc=[20,60,100]

In [ ]:
models = []
ofrs = []
scores = []
lcs = []
for d, m, p in zip (dataset,model,params):
    X_train, X_test, y_train, y_test = getXy(d)
    model, ofr, score,lc = processModel(X_train, X_test, y_train, y_test,m,p,perc)
    gc.collect()
    models.append(model)
    ofrs.append(ofr)
    scores.append(score)
    lcs.append(lc)

In [ ]:
#plt.plot(lc[0,:],'b',lc[1,:],'r')

Let's store them: 

In [ ]:
for m in models:
    m.save()

## Phase 2 (NN for the 1/2 processed dataset)


### Generating 2º dataset processing datasetXB in the previous models

First, let's calculate the pre-stacking dataset: 

In [ ]:
def getPreStackingDataset(dataset, model):
    df = pd.read_csv('datasets/' + dataset + 'B_cleaned.csv')
    df.drop(['Unnamed: 0','Unnamed: 0.1'], inplace=True, axis=1)
    df.set_index(keys=['ID_pair'], inplace=True)

    X = df.drop(labels=['target_imputed'], axis=1).values

    model = LightGradientBoostingRegressionDSBaseModel(model + '0')
    model.load()
    y_pred = model.predict(X)

    df_pred = pd.DataFrame(data=y_pred,columns=['target_predicted'])
    df_pre_stacking = pd.concat(objs=[df.reset_index(),df_pred], axis=1)
    df_pre_stacking_filtered = df_pre_stacking.set_index(keys=['ID_pair','slot'])[['target_imputed','target_predicted']]
    return df_pre_stacking_filtered

In [ ]:
df1 = getPreStackingDataset('dataset1','model1A')

In [ ]:
df2 = getPreStackingDataset('dataset2','model2A')

In [ ]:
df3 = getPreStackingDataset('dataset3','model3A')

In [ ]:
df4 = getPreStackingDataset('dataset4','model4A')

In [ ]:
df5 = getPreStackingDataset('dataset5','model5A')

In [ ]:
df6 = getPreStackingDataset('dataset6','model6A')

Now, let's calculate the real stacking dataset:

In [ ]:
df1_target = df1.reset_index()
df2_target = df2.reset_index()
df3_target = df3.reset_index()
df4_target = df4.reset_index()
df5_target = df5.reset_index()
df6_target = df6.reset_index()

In [ ]:
df_for_stacking = df1_target.merge(
    right=df2_target, on=['ID_pair','slot'],how='outer', suffixes=('_1','_2')
).merge(
    right=df3_target, on=['ID_pair','slot'],how='outer'
).merge(
    right=df4_target, on=['ID_pair','slot'],how='outer', suffixes=('_3','_4')
).merge(
    right=df5_target, on=['ID_pair','slot'],how='outer'
).merge(
    right=df6_target, on=['ID_pair','slot'],how='outer', suffixes=('_5','_6')
)

In [ ]:
df_for_stacking.set_index(keys=['ID_pair','slot'], inplace=True)

In [ ]:
# 4 minutes in process
df_for_stacking['target'] = df_for_stacking[['target_imputed_1','target_imputed_2','target_imputed_3','target_imputed_4','target_imputed_5','target_imputed_6']].apply(lambda x: max([0 if math.isnan(i) else i for i in x]),axis=1)

In [ ]:
df_for_stacking.drop(labels=['target_imputed_1','target_imputed_2','target_imputed_3','target_imputed_4','target_imputed_5','target_imputed_6'], inplace=True, axis=1)

### Cleaning 2º phase dataset 

In [ ]:
#sns.heatmap(~df_for_stacking.isna())

In [ ]:
df_for_stacking_cleaned = cleanStackingDataset(df_for_stacking,['target_predicted_1','target_predicted_2','target_predicted_3','target_predicted_4','target_predicted_5','target_predicted_6'])

In [ ]:
#sns.heatmap(~df_for_stacking_cleaned.isna())

In [ ]:
df_for_stacking_cleaned.to_csv('datasets/dataset_for_stacking_cleaned.csv')

### NN Model 

In [ ]:
dfdnn = pd.read_csv('datasets/dataset_for_stacking_cleaned.csv')

In [ ]:
dfdnn.info()

In [ ]:
dfdnn.set_index(keys=['ID_pair','slot'], inplace=True)

In [ ]:
X = dfdnn.drop(labels=['target'], axis=1).values
y = dfdnn['target'].values.reshape(-1,1)  

In [ ]:
scalerX = MinMaxScaler()
X_s = scalerX.fit_transform(X)
scalery = MinMaxScaler()
y_s = scalery.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_s, y_s, test_size=0.33, random_state=42)

In [ ]:
perc = [25,50,75,100]

In [ ]:
params = DNNRegressionKerasDSBaseParamsToMap(layers=[12,8,4], alpha=0.01, batch_size=64, epochs=10)
dnn = ModelDSBaseWrapper('DNN',X_train,y_train,X_test, y_test,perc,DNNRegressionKerasDSBaseModel,params)

In [ ]:
dnn.train()

In [ ]:
dnnlc=dnn.getLearningCurves()

In [ ]:
plt.plot(dnnlc[0,:],'b',dnnlc[1,:],'r')

In [ ]:
dnn.save()

In [ ]:
joblib.dump(scalerX,'model_persistance/dnn_scalerX.sav')

In [ ]:
joblib.dump(scalery,'model_persistance/dnn_scalery.sav')

# End of Case! 